# 板橋区議会会議録トピックモデリング分析
# Itabashi Ward Council Minutes Topic Modeling Analysis

このノートブックでは、板橋区議会の会議録データに対してトピックモデリング分析を実行します。

## 分析内容
1. データの読み込みと前処理
2. LDA (Latent Dirichlet Allocation) によるトピック抽出
3. トピックの可視化
4. 時系列でのトピック変化分析

In [ ]:
# 必要なライブラリのインポート
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import MeCab
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style('whitegrid')

In [ ]:
# データの読み込み
def load_minutes_data(file_path):
    """会議録JSONデータを読み込む"""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# サンプルデータの読み込み（実際のパスに変更してください）
try:
    minutes_data = load_minutes_data('../crawler/sample/sample_minutes.json')
    print(f"読み込み完了: {len(minutes_data)} 件の会議録")
except FileNotFoundError:
    print("サンプルデータが見つかりません。先にクローラーを実行してください。")
    minutes_data = []

In [ ]:
# データの前処理
class JapaneseTextPreprocessor:
    def __init__(self):
        # MeCabの初期化（システムにMeCabがインストールされている前提）
        try:
            self.mecab = MeCab.Tagger('-Owakati')
        except:
            print("MeCabが利用できません。代替の分かち書き処理を使用します。")
            self.mecab = None
        
        # ストップワード（一般的な助詞、助動詞など）
        self.stop_words = {
            'は', 'が', 'を', 'に', 'で', 'と', 'の', 'から', 'まで', 'より',
            'です', 'である', 'であり', 'であっ', 'でし', 'だっ', 'だ', 'では',
            'という', 'といっ', 'として', 'について', 'により', 'による',
            'こと', 'もの', 'ため', 'ところ', 'そう', 'よう', 'ここ', 'そこ',
            '議長', '議員', 'さん', '委員', '質問', '答弁', '会議', '本日'
        }
    
    def tokenize(self, text):
        """テキストを分かち書きする"""
        if self.mecab:
            tokens = self.mecab.parse(text).strip().split()
        else:
            # 簡易的な分かち書き（MeCabが使えない場合）
            tokens = re.findall(r'[ぁ-んァ-ヶ一-龠]+|[a-zA-Z0-9]+', text)
        
        # ストップワード除去と長さフィルタ
        filtered_tokens = [
            token for token in tokens 
            if token not in self.stop_words and len(token) > 1
        ]
        
        return filtered_tokens
    
    def preprocess_speeches(self, minutes_data):
        """会議録から発言テキストを抽出・前処理"""
        speeches = []
        
        for meeting in minutes_data:
            if 'agenda_items' in meeting:
                for item in meeting['agenda_items']:
                    if 'speech_text' in item:
                        speech = {
                            'text': item['speech_text'],
                            'speaker': item.get('speaker', ''),
                            'agenda': item.get('agenda_item', ''),
                            'meeting_date': meeting.get('meeting_date', ''),
                            'committee': meeting.get('committee', '')
                        }
                        speeches.append(speech)
        
        return speeches

# 前処理の実行
preprocessor = JapaneseTextPreprocessor()
speeches = preprocessor.preprocess_speeches(minutes_data)
print(f"抽出した発言数: {len(speeches)}")

In [ ]:
# トピックモデリングの実行
if len(speeches) > 0:
    # テキストの前処理とベクトル化
    documents = [' '.join(preprocessor.tokenize(speech['text'])) for speech in speeches]
    
    # TF-IDFベクトライザー
    vectorizer = TfidfVectorizer(
        max_features=1000,  # 最大特徴量数
        min_df=2,          # 最小文書頻度
        max_df=0.8,        # 最大文書頻度
        ngram_range=(1, 2) # 1-gram と 2-gram
    )
    
    doc_term_matrix = vectorizer.fit_transform(documents)
    feature_names = vectorizer.get_feature_names_out()
    
    # LDAモデルの学習
    n_topics = 5  # トピック数
    lda_model = LatentDirichletAllocation(
        n_components=n_topics,
        random_state=42,
        max_iter=10
    )
    
    lda_model.fit(doc_term_matrix)
    
    print("トピックモデリング完了")
else:
    print("分析するデータがありません")

In [ ]:
# トピックの可視化
def display_topics(model, feature_names, num_top_words=10):
    """トピックとその主要単語を表示"""
    topics = []
    
    for topic_idx, topic in enumerate(model.components_):
        top_words_idx = topic.argsort()[-num_top_words:][::-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topic_words = ', '.join(top_words)
        
        print(f"トピック {topic_idx + 1}: {topic_words}")
        topics.append({
            'topic_id': topic_idx + 1,
            'words': top_words,
            'weights': topic[top_words_idx]
        })
    
    return topics

if len(speeches) > 0:
    topics = display_topics(lda_model, feature_names)
    
    # トピック分布の可視化
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, topic in enumerate(topics):
        if i < len(axes):
            axes[i].barh(range(len(topic['words'])), topic['weights'])
            axes[i].set_yticks(range(len(topic['words'])))
            axes[i].set_yticklabels(topic['words'])
            axes[i].set_title(f'Topic {topic["topic_id"]}')
            axes[i].invert_yaxis()
    
    # 余ったサブプロットを削除
    for i in range(len(topics), len(axes)):
        axes[i].remove()
    
    plt.tight_layout()
    plt.show()
else:
    print("可視化するデータがありません")

In [ ]:
# 時系列分析（時間があれば）
if len(speeches) > 0:
    # 各文書のトピック分布を取得
    doc_topic_dist = lda_model.transform(doc_term_matrix)
    
    # DataFrameとして整理
    df_speeches = pd.DataFrame(speeches)
    df_speeches['meeting_date'] = pd.to_datetime(df_speeches['meeting_date'], errors='coerce')
    
    # トピック分布を追加
    for i in range(n_topics):
        df_speeches[f'topic_{i+1}_prob'] = doc_topic_dist[:, i]
    
    print("データフレーム作成完了")
    print(df_speeches.head())
else:
    print("時系列分析用のデータがありません")

## 分析結果のまとめ

このノートブックでは以下の分析を実行しました：

1. **データ前処理**: 会議録テキストの分かち書きとストップワード除去
2. **トピックモデリング**: LDAを使用したトピック抽出
3. **可視化**: 各トピックの主要単語を棒グラフで表示
4. **データ構造化**: 分析結果をDataFrameとして整理

### 今後の拡張案
- より多くのデータでの分析
- 時系列でのトピック変化の追跡
- 発言者別の分析
- 委員会別の比較分析
- 感情分析の追加